In [1]:
import folium
from geopy import distance
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
from folium.plugins import MarkerCluster
import pandas as pd

In [5]:
df = pd.read_csv("Airbnb_Texas_Rentals.csv")
airbnb = pd.DataFrame()
airbnb["latitude"] = df["latitude"]
airbnb["longitude"] = df["longitude"]
airbnb["url"] = df["url"]
del df
airbnb = airbnb.dropna(subset = ["latitude"])
airbnb = airbnb.dropna(subset = ["longitude"])
airbnb.head()

,latitude,longitude,url
0,30.020138,-95.293996,https://www.airbnb.com/rooms/18520444?location...
1,29.503068,-98.447688,https://www.airbnb.com/rooms/17481455?location...
2,29.829352,-95.081549,https://www.airbnb.com/rooms/16926307?location...
3,30.637304,-96.337846,https://www.airbnb.com/rooms/11839729?location...
4,32.747097,-97.286434,https://www.airbnb.com/rooms/17325114?location...


In [22]:
def texas():
    
    geolocator = Nominatim(user_agent="Texas")
    
    a = input("Do you want to type the place or the coordinates? [p/c]: ")
    if a == "p":
        place = input("Type the name of the place: ")
        rad = int(input("Type the distance in km: "))
        print("Wait few seconds...")
        location = geolocator.geocode(place)
        loc_input = (location.latitude, location.longitude)
    elif a == "c":
        lat = float(input("Type the latitude: "))
        long = float(input("Type the longitute: "))
        loc_input = (lat, long)
        location = geolocator.reverse(loc_input)
        print("You are looking for a house in this place: " + location.address) 

        rad = int(input("Type the distance in km: "))
        print("Wait few seconds...")

    else:
        return print("Error: digit only 'p' or 'c'")
    

    #location = geolocator.geocode(place)
    
    #loc_input = (location.latitude, location.longitude)
    
    tooltip = "Click me!"
    
    m = folium.Map(
    location = loc_input,
    zoom_start = 6
    )
    
    rad = rad*1000 #convert km in metres
    
    folium.Circle(
    location = loc_input,
    radius = rad, #meter
    color = '#3186cc',
    fill = True,
    fill_color = '#3186cc'
    ).add_to(m)

    folium.Marker(location = loc_input, icon = folium.Icon(color='red')).add_to(m)
    
    marker_cluster = MarkerCluster().add_to(m)
    
    for index, row in airbnb.iterrows():
        point = (row["latitude"], row["longitude"])
        geo = geodesic(point, loc_input).meters

        if geo <= rad:
            folium.Marker(location = (row["latitude"], row["longitude"]),
                          popup = folium.Popup('<a href="'+row["url"]+'"'+'target="_blank"> [Click the link for info] </a>'),



                          tooltip = tooltip
                         ).add_to(marker_cluster)
            
    m.save("texas.html")
    
    return m




In [21]:
texas()

Do you want to type the place or the coordinates? [p/c]: p
Type the name of the place: San Antonio
Type the distance in km: 5
Wait few seconds


In [23]:
texas()

Do you want to type the place or the coordinates? [p/c]: c
Type the latitude: 29.829352
Type the longitute: -95.081549
You are looking for a house in this place: 19317, Rio Villa Drive, Highlands, Harris County, Texas, 77049, USA
Type the distance in km: 5
Wait few seconds...


In [24]:
texas()

Do you want to type the place or the coordinates? [p/c]: s
Error: digit only 'p' or 'c'
